# Inferring the goals of autonomous agents in Gen.jl

Gen.jl is a a runtime system and meta-programming library for compositional probabilistic generative models, inference algorithms, and deep learning. This tutorial will show you how to use Gen.jl to build a probabilistic model of an autonomous agent that has latent beliefs and goals, and use probabilistic inference to infer the agent's goals given their observed behavior. This model is based on research at MIT ProbComp ([Cusumano-Towner et al., 2017](https://arxiv.org/abs/1704.04977)).

In the images below, we see a drone that starts at the location (shown in blue), and we observe its trajectory over time (shown in orange). Our task is to predict the drone's destination. We show the inferred distribution of likely destinations (red) obtained using the probabilistic model presented in this notebook. We show inferences for the same dataset of measured drone locations, but for two different scenes. Do these inferences make sense to you?

<img src="resources/goals/image.png" alt="Drawing" style="width: 800px;"/>



This notebook takes a deeper look at the model behind these inferences, introduces a data visualization methodology for probabilistic programming, and studies how the accuracy of the inferences depend on properties of the model and the amount of computation used.

First, edit the cell below to use the number of parallel processes you want. If you are viewing this on a server provided by ProbComp, use 32 processes.

In [ ]:
addprocs(4);

In [ ]:
import Gen
@everywhere using Gen;

## 1. Modeling an autonomous agent

In Gen.jl, probabilistic models are represented by probabilistic programs. Probabilistic programs are are programmatic descriptions of stochastic generative processes. A probabilistic program in Gen.jl is very much like a regular Julia function, except that certain expressions in the program are labeled with addresses using either `@g(expression, address)` or `@e(expression, address)`, and that the syntax for defining a probabilistic program is slightly different than the syntax for defining a Julia function. 

Below, we define an `agent_model` probabilistic program that models the beliefs, goals, and goal-directed motion of an autonomous agent. The model assumes the agent starts at a given location in a scene, and desires to move to a goal location. We model the motion of the agent by assuming that the agent walks along a path from the goal to its destination that is constructed using a path-planning algorithm based on the rapidly-exploring random tree (RRT) algorithm from robotics. We have implemented the RRT algorithm and simple library for composing scenes in Julia source files.

In [ ]:
@everywhere include("resources/goals/scene.jl")
@everywhere include("resources/goals/path_planner.jl")
@everywhere include("resources/goals/uniform_2d.jl");

In [ ]:
@program agent_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    
    wall_height = 30.
    walls = @e([
        Wall(Point(20., 40.), 1, 40., 2., wall_height),
        Wall(Point(60., 40.), 2, 40., 2., wall_height),
        Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height),
        Wall(Point(20., 80.), 1, 15., 2., wall_height),
        Wall(Point(20., 40.), 2, 40., 2., wall_height) ], "walls")
    for wall in walls
        add!(scene, wall)
    end
    
    # time points at which we observe the agent's location
    observation_times = @e(collect(linspace(0.0, 200.0, 20)), "times")
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = @g(uniform_2d(0, 100, 0, 100), "start")
    
    # the destination of the agent is a random point in the scene
    destination = @g(uniform_2d(0, 100, 0, 100), "destination")
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = @g(normal(locations[i].x, noise), "x$i")
        measured_y = @g(normal(locations[i].y, noise), "y$i")
    end
    
    # record other program state for rendering
    @e(final_path, "final-path")
    @e(scene, "scene")
end;

We can execute this probabilistic program to generate probable scenarios. When we execute a probabilistic program, we record the values of any tagged expressions that were encountered during its execution (e.g. ` "start"`, `"destination"`, etc.) into a **trace**. We create a new empty trace with `ProgramTrace()`:

In [ ]:
trace = ProgramTrace();

We execute a probabilistic program and record its expressions into a trace with `@generate!(program(args), trace)`:

In [ ]:
@generate!(agent_model(), trace)
println("start: ", trace["start"])
println("destination: ", trace["destination"])
println("x1 through x4: ", map((i) -> trace["x$i"], 1:4))
println("y1 through y4: ", map((i) -> trace["y$i"], 1:4))

If we execute the program again it again, we get a different result:

In [ ]:
@generate!(agent_model(), trace)
println("start: ", trace["start"])
println("destination: ", trace["destination"])
println("x1 through x4: ", map((i) -> trace["x$i"], 1:4))
println("y1 through y4: ", map((i) -> trace["y$i"], 1:4))

We can also print the whole trace. For now, don't worry about constraints, interventions, or proposals. Note that the `recorded` section lists the values that each of the named expressions took during the execution:

In [ ]:
print(trace)

# 2. Visualizing the probabilistic behavior of a model using a trace rendering

Printing a trace is not a very good way to understand the probabilistic behavior of a program. Instead, we use a **trace rendering** to produce a visual representation of the trace. The trace rendering encodes the trace into a representation that the human visual system can quickly interpret. In Gen.jl a trace renderer is simply object that has a method "`render(renderer, trace::Trace)`". In Jupyter notebooks, we render traces using JavaScript code that renders traces onto a Document Object Model (DOM) element in the output of notebook cells.

In the file included below, we define a trace renderer for `agent_model` using JavaScript and CSS. In particular we use the Javascript data visualization library [D3](https://d3js.org/). This cell returns CSS and Javascript code into its output cell, which gets added to the DOM by Jupyter. Inside this file, we define the trace renderer in Javascript, and register it with Gen using the Javascript function `Gen.register_jupyter_renderer`. Each Javascript trace renderer is given a name when it is registered, so that the Julia code can send the trace data to the right renderer. A Javascript trace renderer is a function with signature `function(dom_id, trace, configuration) {.. }`. The first argument `dom_id` defines what element of the DOM the renderer should write to, and `trace` contains the trace data sent from Julia. The Javascript code in the cell below should have correct Javascript syntax highlighting (you may have to click in the cell to activate the syntax highlighting).

In [ ]:
include("resources/goals/rendering.jl")

We create a DOM element that the trace renderer renders to using the `Figure` function in Julia. The `here(figure)` places a DOM element in the output of the cell onto which Javascript trace renderers will be able to render traces. It should appear as as a mostly empty output area.

In [ ]:
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20,
                titles=["Trace"])
here(figure)

Now, we construct a Julia trace renderer object that will delegate rendering to the Javascript trace renderer that we defined above. We attach the renderer to the first viewport in the figure above.

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict())
attach(renderer, id(figure => 1));

We then execute the program, and render the resulting trace. You should see the trace rendering appear on the left side of the rendering area above. Run the cell below a few times to see different executions of the program.

In [ ]:
@generate!(agent_model(), trace)
render(renderer, trace)

We also wrote a renderer that just shows the legend. We ask that renderer to put a legend in the right side of the rendering area above.

In [ ]:
legend_renderer = JupyterInlineRenderer("agent_model_legend_renderer", Dict())
attach(legend_renderer, id(figure => 2))
render(legend_renderer, trace)

A probabilistic program is stochastic, and the distribution over its executions defines a probability distribution over potential scenarios. Let's get a sense of this probability distribution by rendering many executions, one after another, in an animation:

In [ ]:
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20,
                titles=["Trace"])
here(figure)

In [ ]:
attach(legend_renderer, id(figure => 2))
render(legend_renderer, trace)
attach(renderer, id(figure => 1))
for i=1:100
    trace = ProgramTrace()
    @generate!(agent_model(), trace)
    render(renderer, trace)
    sleep(0.1)
end

We can also render many runs side by side in a grid, instead of an animation. This also gives a sense of the probability distribution represented by the probabilistic program.

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
for i=1:12
    trace = ProgramTrace()
    @generate!(agent_model(), trace)
    attach(renderer, id(figure => i))
    render(renderer, trace)
end

A key feature of Gen.jl is the ability to constrain the values of random choices that a probabilistic program makes during the course of its execution. Let's see how we can modify the behavior of the program if we constrain the starting location of the agent and its destination location using the `constrain!(trace, name, value)` method. We will constrain the start location to lie in the upper left corner and the destination to lie in the bottom right corner.

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
for i=1:12
    trace = ProgramTrace()
    constrain!(trace, "start", Point(10, 10))
    constrain!(trace, "destination", Point(90, 90))
    @generate!(agent_model(), trace)
    attach(renderer, id(figure => i))
    render(renderer, trace)
end

Now we have a sense of the variability in the paths that the agent takes.

We can also get a sense for the variability by overlaying many traces on top of one another. Our renderer supports this as part of its configuration:

In [ ]:
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20,
                titles=["Trace"])
here(figure)

In [ ]:
attach(legend_renderer, id(figure => 2))
render(legend_renderer, trace)
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("path" => "overlay"))
attach(renderer, id(figure => 1))
CSS("#$(id(figure)) .path { visibility: hidden; }")

In [ ]:
for i=1:20
    trace = ProgramTrace()
    intervene!(trace, "start", Point(10, 10))
    intervene!(trace, "destination", Point(90, 90))
    @generate!(agent_model(), trace)
    render(renderer, trace)
end

# 3. Probabilistic inference

So far, we have generated scenarios that the model thinks are probable, and we have constrained some random choices and simulated the consequence. However, suppose we had observed the start location of the agent, and a certain sequence of locations of the agent along its path, and we wanted to know probable goal locations?

This is a query that cannot be answered simply by forward simulation of the program, because the location of the agent is a *consequence* and not a *cause* of the destination. We can easily find probable consequences given the causes as we did above, but finding probable causes given the consequences requires a bit more work.

Here is an example dataset showing measured locations for the first 7 time points:

In [ ]:
dataset = [
    Point(10.3867,10.3889)
    Point(11.0188,20.0843)
    Point(11.6663,30.0142)
    Point(11.4507,41.5159)
    Point(13.8081,53.0258)
    Point(13.1958,61.3572)
    Point(17.1566,73.7131)
]

The first step in inference is to constrain the random choices that are observed.

In [ ]:
trace = ProgramTrace()
constrain!(trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end

Now, when we run the program in this trace, we find that the score of the trace tells us how well the other values in the trace match the value of the constraints. Very negative scores that the other values in the trace, such as the destination and the projected path, do not match well with the constrained path points. The path points that were constrained are identified in the trace renderings by a dashed border.

In [ ]:
figure = Figure(num_rows=3, num_cols=3, width=900, height=900, trace_width=100, trace_height=100)
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .score { visibility: visible; }
""")

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict())
for i=1:30
    t = deepcopy(trace)
    (score, _) = @generate!(agent_model(), t)
    attach(renderer, id(figure => i))
    render(renderer, t; score=score)
end

Given a large number of traces generated by the program, and a score for each trace that tells us how well that trace matches the constraints, we can select those trace that match the constraints. These traces then represent plausible hypotheses that can explain the observed data. This is the approach taken by the two basic inference algorithms that we implement next, *Importance Sampling* and *Metropolis-Hastings*.

### Importance sampling

The code cell below shows a basic importance sampling procedure that uses the score of traces to probabilistically filter them and tries to select a trace that explains the data well. Note that this procedure is a regular Julia function.

In [ ]:
function agent_model_importance_sampling(trace::ProgramTrace, num_samples::Int)
    
    # the 'trace' argument contains the constraints that represent the 
    # observed dataset. 
    
    # perform many independent executions of the probabilistic program, and 
    # record each execution in a trace. store the 'score' for each trace, 
    # which encodes how well the trace matches the constraints
    traces = Vector{ProgramTrace}(num_samples)
    scores = Vector{Float64}(num_samples)
    for k=1:num_samples
        t = deepcopy(trace)
        (scores[k], _) = @generate!(agent_model(), t)
        traces[k] = t
    end
    
    # compute a reltaive weight for each trace indicating how well the trace
    # matches the constraints relative to the other traces generated. A large
    # weight indicates that the trace matches the constraints well, and may be
    # a better explanation for the observed data than other trace.
    
    # weights = exp.(scores) / sum(exp.(scores))
    weights = exp.(scores - logsumexp(scores))
    weights = weights / sum(weights)
    
    # pick a trace in propotion to its relative weight and return it
    chosen = rand(Categorical(weights))
    return traces[chosen]
end;

Because this Julia function treats the execution traces of the probabilistic program `agent_model` as data, this inference procedure is an example of a [meta-program](https://en.wikipedia.org/wiki/Metaprogramming). In Gen.jl, inference is done using meta-programs that reason about the traces of other programs. This is in contrast to most other probabilistic programming systems, in which users do not write their own custom meta-programs, but instead rely on built-in generic inference techniques.

Let's see the output of our importance sampling procedure. Recall that each run of the procedure produces one trace of the `agent_model` probabilistic program. Note that the importance sampling procedure is stochastic---repeated executions will generate different traces. This is an important feature---we want inference algorithms to give us probable explanations for the data, but also a sense of how uncertain we are in those explanations. By executing the inference algorithm many times, we can get a sense of the uncertainty in the inferences.

In [ ]:
num_samples_list = [1, 4, 16]
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n samples)", num_samples_list))
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .path.constrained { visibility: visible; }
    #$(id(figure)) .path_segments { visibility: hidden; }
    #$(id(figure)) .destination { fill-opacity: 0.5; }
""")

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("destination" => "overlay"))

trace = ProgramTrace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, id(figure => i))
    title =  "SIR ($num_samples particles)"
    for j=1:100
        output_trace = agent_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

We ran the inference algorithm using `num_samples=1`, `num_samples=4`, and `num_samples=16`. Note that when we increase `num_samples` we get more plausible traces as output. For `num_samples=16`, the algorithm tells us that it thinks the agent is headed either into the box or in the bottom left or bottom of the scene. Also note that the more plausible answers come at a greater computation cost---we have to wait longer for each new sample to arrive. Understanding the time-accuracy tradeoffs in approximate probabilistic inference algorithms is an active area of research (see [Cusumano-Towner, Mansinghka 2016](https://arxiv.org/abs/1705.07224) for recent work in this area).

### Metropolis-Hastings Inference

Here, we show a basic Metropolis-Hastings inference allgorithm written in Gen.jl that operates according to similar principles. Instead of generating a bunch of traces and then scoring each and selecting it as output, Metropolis-Hastings generates traces one by one and decides to stochastically replace the previous trace with a new trace if the new trace has a higher score than the previous trace.

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict())
figure = Figure(num_rows=1, num_cols=3, width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=["proposed trace", "current trace"])
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .score { visibility: visible; }
""")

In [ ]:
attach(legend_renderer, id(figure => 3))
render(legend_renderer, trace)
current_trace = ProgramTrace()
intervene!(current_trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset)
    constrain!(current_trace, "x$i", point.x)
    constrain!(current_trace, "y$i", point.y)
end
(current_score, _) = @generate!(agent_model(), current_trace)
for i=1:100
    proposed_trace = deepcopy(current_trace)
    (proposed_score, _) = @generate!(agent_model(), proposed_trace)
    if log(rand()) < proposed_score - current_score
        current_trace = proposed_trace
        current_score = proposed_score
    end
    attach(renderer, id(figure => 1))
    render(renderer, proposed_trace, score=proposed_score)
    attach(renderer, id(figure => 2))
    render(renderer, current_trace, score=current_score)
    sleep(0.1)
end

### Changing the scene

Next, we change the scene in the model by adding a door in the top wall. We do this by intervening on the "walls" variable in the `agent_model` probabilistic program. We use the same dataset, and compare the inferences when the model assumes the original scene (without the door) and the modified scene (with the door). 

In [ ]:
figure = Figure(num_rows=1, num_cols=2, width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=["without door", "with door"])
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .path.constrained { visibility: visible; }
    #$(id(figure)) .path_segments { visibility: hidden; }
    #$(id(figure)) .destination { fill-opacity: 0.5; }
""")

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("destination" => "overlay"))
num_samples = 64
num_approximate_samples = 50
trace = ProgramTrace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end

# scene without door
attach(renderer, id(figure => 1))
for j=1:num_approximate_samples
    output_trace = agent_model_importance_sampling(trace, num_samples)
    render(renderer, output_trace)
end

# scene with door
wall_height = 30.
new_walls = [
    Wall(Point(60.-15., 40.), 1, 15. + 2., 2., wall_height),
    Wall(Point(20., 40.), 1, 15., 2., wall_height),
    Wall(Point(60., 40.), 2, 40., 2., wall_height),
    Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height),
    Wall(Point(20., 80.), 1, 15., 2., wall_height),
    Wall(Point(20., 40.), 2, 40., 2., wall_height) 
]
intervene!(trace, "walls", new_walls)
attach(renderer, id(figure => 2))
for j=1:num_approximate_samples
    output_trace = agent_model_importance_sampling(trace, num_samples)
    render(renderer, output_trace)
end

When there is no upper door, inference tells us that it is probable that the agent is headed into the room. When there is an upper door, inference tells us it is unlikely that the agent is headed into the room. Intuitively, this is because the model assumes that the agent takes relatively direct paths from its start location to its destination. If the agent were headed into the room, it would have taken the more direct route through the top door.

# 4. Improving the model

The probabilistic model `agent_model` used above made a lot of assumptions that are unlikely to hold in the real world. For example, the agent always takes pretty direct paths from its starting location to its final destination. What if the agent is more unpredictable? What if it takes detours?

Here is a dataset that has a detour in it, and that does not match our model's expectations.

In [ ]:
detour_dataset = [
    Point(9.59825,8.92063)
    Point(21.8936,9.54817)
    Point(30.9534,10.8819)
    Point(43.1137,9.75395)
    Point(48.8929,10.4189)
    Point(46.0282,21.7662)
    Point(35.0281,25.9994)
    Point(27.2084,33.5729)
    Point(20.1662,39.9398)
    Point(18.7309,50.0026)
];

In [ ]:
figure = Figure(num_cols=2, width=900, height=400, trace_width=100, trace_height=100, margin_top=20)
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .path.constrained { visibility: visible; }
    #$(id(figure)) .path_segments { visibility: hidden; }
    #$(id(figure)) .destination { visibility: hidden; }
""")

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("destination" => "overlay"))
attach(renderer, id(figure => 1))

# show the dataset above.
trace = ProgramTrace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(detour_dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
@generate!(agent_model(), trace)

render(renderer, trace)

# show a legend
attach(legend_renderer, id(figure => 2))
render(legend_renderer, trace)

Let's see what happens when we try to do probabilistic inference given this dataset, using the `agent_model` model. We will use the same importance sampling algorithm we wrote above, with the same settings for the  `num_samples` parameter.

In [ ]:
num_samples_list = [1, 4, 16]
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n samples)", num_samples_list))
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .path.constrained { visibility: visible; }
    #$(id(figure)) .path_segments { visibility: hidden; }
    #$(id(figure)) .destination { fill-opacity: 0.5; }
""")

In [ ]:
trace = ProgramTrace()
intervene!(trace, "start", Point(10, 10))
for (i, point) in enumerate(detour_dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end

num_approximate_samples = 50
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, id(figure => i))
    for j=1:num_approximate_samples
        output_trace = agent_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

The inferences do not look intuitive. This is because the model `agent_model` cannot explain the detour. It assumes that the detour must be the goal, an it explains the observed data as a very unlikely accident of noise. This is an example of **model mis-specification**.  In order to make reasonable inferences for datasets that may contain a detour, we need to improve our model. Here is a new probabilistic program that models an agent that may or may not use an arbitrary waypoint (e.g. detour) when planning its path from its starting location to its destination:

In [ ]:
@everywhere @program agent_waypoint_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    
    wall_height = 30.
    walls = @e([
        Wall(Point(20., 40.), 1, 40., 2., wall_height)
        Wall(Point(60., 40.), 2, 40., 2., wall_height)
        Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height)
        Wall(Point(20., 80.), 1, 15., 2., wall_height)
        Wall(Point(20., 40.), 2, 40., 2., wall_height) ], "walls")
    for wall in walls
        add!(scene, wall)
    end
    
    # time points at which we observe the agent's location
    observation_times = @e(collect(linspace(0.0, 200.0, 20)), "times")
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = @g(uniform_2d(0, 100, 0, 100), "start")
    
    # the destination of the agent is a random point in the scene
    destination = @g(uniform_2d(0, 100, 0, 100), "destination")
    
    if @g(flip(0.5), "use-waypoint")
        waypoint = @g(uniform_2d(0, 100, 0, 100), "waypoint")
        (tree1, rough_path1, final_path1) = plan_path(start, waypoint, scene)
        (tree2, rough_path2, final_path2) = plan_path(waypoint, destination, scene)
        
        # if either path planner sub-problem failed, then no path was found (final_path is null)
        if isnull(final_path1) || isnull(final_path2)
            final_path = Nullable{Path}() # null
        else
            final_path = Nullable{Path}(concatenate(get(final_path1), get(final_path2)))
        end
    else
        (tree, rough_path, final_path) = plan_path(start, destination, scene)
    end
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    
    if @e(isnull(final_path), "planning-failed")
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = @g(normal(locations[i].x, noise), "x$i")
        measured_y = @g(normal(locations[i].y, noise), "y$i")
    end
    
    # record other program state for rendering
    @e(final_path, "final-path")
    @e(scene, "scene")
end;

Here are some simulations from the program, for a constrained start and destination that both lie one side of the scene. The waypoint, when there is one, is shown as a pink circle:

In [ ]:
figure = Figure(num_rows=3, num_cols=3, width=900, height=900, trace_width=100, trace_height=100)
here(figure)

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict())
traces = []
for i=1:18
    trace = ProgramTrace()
    constrain!(trace, "start", Point(10, 10))
    constrain!(trace, "destination", Point(30, 90))
    @generate!(agent_waypoint_model(), trace)
    attach(renderer, id(figure => i))
    render(renderer, trace)
    push!(traces, trace)
end

Notice that sometimes, the path has a clear waypoint/detour whereas other times it does not. Run the above cell a few times if you don't initially see a clear waypoint. The current trace rendering does not show the waypoint explicitly, but it could be extended to do so.

Let's do inference in this new model on the detour dataset. We'll use importance sampling again. The only change in this procedure from the earlier importance sampling procedure is the choice of model (`agent_waypoint_model` instead of `agent_model`).

In [ ]:
@everywhere function agent_waypoint_model_importance_sampling(trace::ProgramTrace, num_samples::Int)
    traces = Vector{ProgramTrace}(num_samples)
    scores = Vector{Float64}(num_samples)
    for k=1:num_samples
        t = deepcopy(trace)
        (scores[k], _) = @generate!(agent_waypoint_model(), t)
        traces[k] = t
    end
    weights = exp.(scores - logsumexp(scores))
    weights = weights / sum(weights)
    chosen = rand(Categorical(weights))
    return traces[chosen]
end

In [ ]:
num_samples_list = [1, 4, 16]
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n) -> "SIR ($n samples)", num_samples_list))
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .path.constrained { visibility: visible; }
    #$(id(figure)) .path_segments { visibility: hidden; }
    #$(id(figure)) .destination { fill-opacity: 0.5; }
    #$(id(figure)) .waypoint { visibility: hidden; }
""")

In [ ]:
trace = ProgramTrace()
constrain!(trace, "start", Point(10, 10))
for (i, point) in enumerate(detour_dataset)
    constrain!(trace, "x$i", point.x)
    constrain!(trace, "y$i", point.y)
end
renderer = JupyterInlineRenderer("agent_model_renderer", Dict("destination" => "overlay"))
num_approximate_samples = 50
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, id(figure => i))
    for j=1:num_approximate_samples
        output_trace = agent_waypoint_model_importance_sampling(trace, num_samples)
        render(renderer, output_trace)
    end
end

It looks like the results are not accurate. Let's try more computation. We will try many more particles, and we will parallelize across the 32 cores on this instance, but it will still take a minute or two to run. Note that since we are parallelizing with `pmap`, the samples will appear all at once and not incrementally.

Please the number of samples below to be `[64, 256, 1024]`:

In [ ]:
num_samples_list = [64];

In [ ]:
figure = Figure(num_rows=1, num_cols=length(num_samples_list),
                width=900, height=300, trace_width=100, trace_height=100,
                margin_top=20, titles=map((n)-> "SIR ($n samples)", num_samples_list))
here(figure)

In [ ]:
CSS("""
    #$(id(figure)) .path.recorded { visibility: hidden; }
    #$(id(figure)) .path.constrained { visibility: visible; }
    #$(id(figure)) .path_segments { visibility: hidden; }
    #$(id(figure)) .destination { fill-opacity: 0.5; }
    #$(id(figure)) .waypoint { visibility: hidden; }
""")

In [ ]:
num_approximate_samples = 50
for (i, num_samples) in enumerate(num_samples_list)
    attach(renderer, id(figure => i))
    
    # parallel map
    output_traces = pmap(agent_waypoint_model_importance_sampling,
                          [trace for _ in 1:num_approximate_samples],
                          [num_samples for _ in 1:num_approximate_samples])
    for output_trace in output_traces
        render(renderer, output_trace)
    end
end

The extra computation seems to have given us reasonable inferences. Recall that inference in the original model with a typical dataset only required 16 particles to give reasonalbe results for a particular dataset. Why do we need to do more computation to get accurate inferences for `agent_waypoint_path` than we needed for `agent_model`? This is because a random execution of the improved model is a lot less likely to match a typical dataset generated from it than a random execution of the original model was to match a typical dataset generated from it. Conceptually, the distribution on datasets produced by `agent_waypoint_path` has more entropy than the distribution on datasets produced by `agent_path`. This means we need to increase the number of samples to incrase the probability that we get one that matches the data.